# Experiments: Fooling Lime


## Imports

In [ ]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

## Loading Data and Classifier

In [ ]:
model_type = "emb-att"
nwords

current_model = get_model(model_type, nwords, EMB_SIZE, HID_SIZE, ntags)
model.load_state_dict(torch.load(DATA_MODELS_PATH + 'model_' + task + suffix + '_seed=' + str(seed) + '_coeff='
                                     + str(coeff) + '_num-train=' + str(num_train) + '.pt'))


### Training (with Attention)


In [ ]:
# seeds = [1, 13, 42]
# original seeds taken from their sh file
seeds = [1, 5]

coefficients = [0.0, 1.0, 0.1]

# tasks = ['copy', 'reverse-copy', 'binary-flip', 'en-hi', 'en-de']
# original tasks taken from their sh file

tasks = ['copy', 'reverse-copy', 'binary-flip', 'en-de']

epochs = 30
batch_size = 128

for seed in seeds:
    for coeff in coefficients:
        for task in tasks:
            train(task, epochs, coeff, seed, batch_size, attentions[0])

### Training Baselines (without Attention)
The authors ran experiments with uniform and no attention (ablation studies) and no penalty on impermissible words (loss coefficient 0.0).

In [ ]:
# original seeds taken from their sh file
seeds = [6, 10]

for seed in seeds:
    for task in tasks:
        train(task, epochs, 0.0, seed, batch_size, attentions[1:])

### Results (with Attention)

- load pretrained models which we have saved and included in github repo / code
- visualize some results

### Results Baselines (without Attention)
- load pretrained models which we have saved and included in github repo / code
- visualize some results

## English-German Translation